# [모듈 1.1] CodeCommit을 하면 EventBridge를 통해 CodePipeline 수행

이 노트북은 CodeCommit 레파지토리에 'SageMaker Pipeline config 파일(sm_pipeline_train_config.json)'을 수정 후 push 하면, EventBridge를 통해 CodePipeline을 실행하는 노트북 입니다.

## 참고 자료
- Amazon EventBridge 유저 가이드
   (https://docs.aws.amazon.com/eventbridge/latest/userguide/eb-what-is.html)
- Boto3 Docs : EventBridge
   (https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/events.html)



# 1. 환경 확인

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import sagemaker
import boto3
sys.path.append('./src')

%store -r s3_input_data_uri
print("s3_input_data_uri: n", s3_input_data_uri)


# 2. 리파지토리 로컬에 클로닝

### repository_url :
```
repository_url =  "https://git-codecommit.<REGION>.amazonaws.com/v1/repos/ncf-training-repo"
```

In [ ]:
## Codecommit Repo Name (ncf-training-repo)
repository_name = "ncf-training-repo"


## 4_sm-train-codepipeline에서 만든 훈련 용 repository name 로딩

#%store -r training_repository_name
#repository_name = training_repository_name

print(repository_name)

In [ ]:
repository_url = boto3.client('codecommit').get_repository(repositoryName = repository_name).get('repositoryMetadata').get('cloneUrlHttp')
print(repository_url)

# repository_url = "<Repository URL>"

### git clone 과정에서 아래와 같은 에러 발생시 트러블 슈팅
[에러 처리] CalledProcessError: Command 'b'\nrepository_url=$1\nrepository_name=$2 .......
--> 솔루션 : Kernel Restart 후 노트북 처음부터 다시 실행시켜 주세요.

In [ ]:
%%sh -s {repository_url} {repository_name}

repository_url=$1
repository_name=$2

rm -rf $repository_name

git clone $repository_url 

# 3. 파라미터 세팅

In [ ]:

# code pipeline name, arn, role_arn
codepipeline_name1 = "ncf-code-pipeline-training-sm-pipeline"
codepipeline_arn = boto3.client('codepipeline').get_pipeline(name = codepipeline_name1).get('metadata').get('pipelineArn')
eventbrideexcrole_arn = boto3.client('iam').get_role(RoleName = 'MLOps-EventBridgeRole').get('Role').get('Arn')

# EventBridge Rule Name
eventbridge_rule_name1 = "codepipelinerule1"

# Training data path (Prefix)
data_prefix = "NCFModel/data2"

# EventBridge Rule의 EventPattern json 파일 경로
eventpattern_json = f'src/train_eventpattern.json'


# 4. EventBridge Rule 정의

## 4.1 EventPattern 정의

In [ ]:
# Codecommit Arn
repository_arn = boto3.client('codecommit').get_repository(repositoryName = repository_name).get('repositoryMetadata').get('Arn')
print(repository_arn)

In [ ]:
eventpattern_dict = {
    "source": ["aws.codecommit"],
    "detail-type": ["CodeCommit Repository State Change"],
    "resources": [repository_arn],
    "detail": {
            "referenceType": ["branch"],
            "referenceName": ["master"]
    }
}

print(eventpattern_dict)

#### 4.1.1 설정 딕셔너리 Json 파일로 저장 후 로딩

In [ ]:
import json

eventclient = boto3.client('events')

eventresponse = eventclient.put_rule(
    Name = eventbridge_rule_name1,
    EventPattern = json.dumps(eventpattern_dict)
)


In [ ]:
eventresponse

## 4.2 Event Target 설정

In [ ]:
eventresponse = eventclient.put_targets(
    Rule = eventbridge_rule_name1,
    Targets = [
        {
            'Arn' : codepipeline_arn,
            'Id' : codepipeline_name1,
            'RoleArn' : eventbrideexcrole_arn
        }
    ]
)


In [ ]:
eventresponse

# 5. 새로운 SageMaker Pipeline (Training) 구성 파일 준비

## 5.1. 변수 설정 및 dict 로 저장

In [ ]:
from common_utils import save_json, load_json

# 레포지토리의 sm_pipeline_train_config.json 파일 위치 지정
sm_pipeline_train_config_json_path = repository_name+"/pipelines/ncf/src/sm_pipeline_train_config.json"

# sm_pipeline_train_config.json 파일 내용을 Load
sm_train_pipeline_dict = load_json(sm_pipeline_train_config_json_path)


In [ ]:
from datetime import datetime
currentDateAndTime = datetime.now()

# 현재 시간을 Json 에 넣기 위함
currentTime = currentDateAndTime.strftime("%Y-%m-%d-%H-%M-%S")
print("The current time is", currentTime)

In [ ]:
# sm_pipeline_train_config.json 파일에서 훈련 데이터 위치(s3 uri)와 현재 시간만 업데이트
sm_train_pipeline_dict['s3_input_data_uri'] = s3_input_data_uri
sm_train_pipeline_dict['update_time'] = currentTime

print(sm_train_pipeline_dict)

## 5.2. 설정 딕션너리 Json 파일로 저장 후 로딩

In [ ]:
import json
from common_utils import save_json, load_json

save_json(sm_pipeline_train_config_json_path, sm_train_pipeline_dict)
sm_pipeline_train_dict = load_json(sm_pipeline_train_config_json_path)
print (json.dumps(sm_pipeline_train_dict, indent=2))

# 6. 리파지토리에 푸시 (이벤트 트리거)

In [ ]:
%%sh -s {repository_name} 
repository_name=$1
cd $repository_name
date >> pushtime.log
echo $PWD
git add .
git commit -m "Add new files"
git push origin master


# 7. 변수 저장

In [ ]:
%store eventbridge_rule_name1
%store codepipeline_name1

# 8. EventBridge Rule 및 CodePipeline 실행 확인 (콘솔 화면)

## 9.1 EventBridge Rule 생성 확인
#### 하단에 codepipeline1이 생성된 것을 확인할 수 있습니다.

![eventbridge1.png](img/eventbridge1.png)

## 9.2 codepipelinerule1 의 Event pattern 확인

![eventbridge1-eventpattern.png](img/eventbridge1-eventpattern.png)

## 9.3 codepipelinerule1의 Target 확인

![eventbridge1-target.png](img/eventbridge1-target.png)

## 9.4 EventBridge를 통한 CodePipeline 실행 확인

![codepipeline-training-inprogress1.png](img/codepipeline-training-inprogress1.png)